In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import matplotlib.pyplot as plt
import math
import cv2

In [ ]:
# 실습에 사용할 이미지 로딩
gray_lena_img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Image_Processing_2025_1/lena.bmp',cv2.IMREAD_GRAYSCALE) #처리하기 편하게 흑백이미지를 불러온다

In [ ]:
#1) Design a function to generate the binary image.
#Input : image, threshold
#Output : binary image

def make_binary_img(image, threshold):
  binary_image = np.zeros(image.shape) # 이진화 이미지 담을 이미지 image.copy()라고 해도 됨
  for i in range(image.shape[0]): #
    for j in range(image.shape[1]):
      if image[i][j] >= threshold:
        binary_image[i][j] = 255
      else:
        binary_image[i][j] = 0
  return binary_image

#make_binary_img(gray_lena_img, 128)
plt.imshow(make_binary_img(gray_lena_img, 128),cmap='gray')
plt.show()


In [ ]:
#1번 교수님 버전
def make_binary_img(img, T):
  binary_image = img.copy()

  binary_image[img>=T]=1
  binary_image[img<T]=0

  return binary_image

plt.imshow(make_binary_img(gray_lena_img, 128),cmap='gray')
plt.show()

In [ ]:
#2) Design a function to find the histogram for given input gray image.
#Input : image, number of bins (N)
#Output : histogram (Nx1 vector)

def make_histogram(image, N):
  histogram = np.zeros(N)
  for i in range(image.shape[0]):
    for j in range(image.shape[1]):
      histogram[image[i][j]] += 1
  return histogram

hist_of_img=make_histogram(gray_lena_img, 256)
x=np.arange(0,256,1)
plt.bar(x,hist_of_img)
plt.show()

In [ ]:
# 2번 교수님 버전
# 넘파이 내장 함수는 자유롭게 사용 가능
gray_lena_img_1D = gray_lena_img.flatten()
bins=np.arange(0,256,1)
hist_y, hist_x = np.histogram(gray_lena_img_1D, bins)
plt.bar(bins[:-1],hist_y)
plt.show()

In [ ]:
#3) (Option) Design the Otsu algorithm.
#Input : image
#Output : binary image
# prompt: design the otsu algorithm without opencv. input: image, output: binary image

import matplotlib.pyplot as plt
def otsu_thresholding(image):

  hist = make_histogram(image, 256)  # Calculate the histogram
  total_pixels = image.shape[0] * image.shape[1]
  current_max_variance = 0
  optimal_threshold = 0

  for threshold in range(1, 256):
    # Calculate the number of pixels in each class
    foreground_pixels = sum(hist[:threshold])
    background_pixels = total_pixels - foreground_pixels

    if foreground_pixels == 0 or background_pixels == 0:
      continue

    # Calculate the mean of each class
    foreground_mean = sum([i * hist[i] for i in range(threshold)]) / foreground_pixels
    background_mean = sum([i * hist[i] for i in range(threshold, 256)]) / background_pixels

    # Calculate the between-class variance
    variance = (foreground_pixels / total_pixels) * (background_pixels / total_pixels) * (foreground_mean - background_mean) ** 2

    if variance > current_max_variance:
      current_max_variance = variance
      optimal_threshold = threshold

  binary_image = make_binary_img(image, optimal_threshold)
  return binary_image

# Example usage:
binary_image_otsu = otsu_thresholding(gray_lena_img)

plt.imshow(binary_image_otsu, cmap='gray')
plt.show()


In [ ]:
#4) Design a function to compute MSE and PSNR.
#Input : image1, image2
#Output : MSE, PSNR

img1 = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Image_Processing_2025_1/Image/babbon.bmp',cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Image_Processing_2025_1/Image/pepper.bmp',cv2.IMREAD_GRAYSCALE)

def psnr_func(image1,image2): # 2개의 이미지의 크기기 같아야 됨
    mse=np.mean((image1-image2)**2) #mse 계산식
    psnr=20*math.log10(255/math.sqrt(mse)) #psnr 계산식
    return mse, psnr
val1,val2=psnr_func(img1,img2)

print("MSE : ",val1," PSNR : ",val2)


In [ ]:
#5) Generate 𝑀 × 𝑁 random noisy image.
#Input : 𝑀,𝑁, type_of_noise = uniform or Gaussian
#Output : 𝑀 ×𝑁 noise
#uniform_noise = np.random.random((512,512)) # uniform 노이즈(0~1) 항상 평균인 0.5를 빼주고 곱해야 항상 균일하게 유지하면서 노이즈를 생성함
#gaussian_noise = np.random.normal()

# mean=0,sigma^2인 가우시안 노이즈를 이미지에 추가
def add_gaussian_noise(image, sigma):
    row, col = image.shape
    # uniform_noise = (np.random.random((row, col))-0.5)*100 # uniform 노이즈 추가
    mean=0
    gaussian_noise = np.random.normal(mean,sigma,(row,col)) # gaussian 노이즈 추가
    noisy_image = image + gaussian_noise
    noisy_image = np.clip(noisy_image, 0, 255)
    return noisy_image.astype(np.uint8)

gaus_noise_lena=add_gaussian_noise(gray_lena_img, 50)
plt.title('gaussian noise lena')
plt.imshow(gaus_noise_lena,cmap='gray')
plt.show()

In [ ]:
#6) Compute MSE and PSNR for noisy image.
# psnr의 정상적인 값은 30 근방이다
val1,val2=psnr_func(gray_lena_img,gaus_noise_lena)
print("MSE : ",val1," PSNR : ",val2)